## 멋쟁이 사자처럼 AI School 7기 데일리 키워드 추출 및 워드클라우드 생성기

In [ ]:
# Import and Setting
!pip install sentence_transformers konlpy
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from collections import Counter
# For Word Cloud
from wordcloud import WordCloud
# For konlpy & TA/NLP
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
# For Goolge Sheet
import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
# Load Model
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
# Sheet Load and Preprocessing
sheet_name = input("응답 시트 파일명 (해당 공유 폴더가 내 드라이브에 있어야함!): ")
worksheet = gc.open(sheet_name).sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows)
df.fillna("-")
df = df.iloc[:, -1][1:]
temp = df.str.split("\n")
temp = list(itertools.chain(*temp))
intergrated = "\n".join(temp)

In [ ]:
# KeyBERT
okt = Okt()

tokenized_doc = okt.pos(intergrated)
tokenized_nouns = " ".join([word[0] for word in tokenized_doc if (word[1]=="Noun" or word[1]=="Alpha")])

n_gram_range = (1, 1)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

doc_embedding = model.encode([intergrated])
candidate_embeddings = model.encode(candidates)

top_n = int(input("몇개의 키워드를 추출할 것인지?: "))
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [ ]:
# Word Cloud
wc = WordCloud(
    font_path = "/content/drive/MyDrive/malgun.ttf",
    width=500,
    height=500,
    max_font_size=250,
    scale=1.2,
    background_color="black"
).generate_from_frequencies(Counter(str(tokenized_nouns).split(" ")))

In [ ]:
# Output
plt.figure(figsize=(15, 15))
plt.imshow(wc)
plt.axis("off")

for kw in keywords:
    print(kw, end=" ")